# Install Libraries

In [ ]:
!pip install folium

# Import Libraries

In [ ]:
import pandas as pd
import geopandas as gpd
import json
import numpy as np
import math
import folium

# Read CSV Files

Read the CSV file with data about the capacities in public transportation vehicles

In [ ]:
gefaessgroesse_df = pd.read_csv('data/gefaessgroesse.csv', delimiter=';',encoding = "utf-8")
gefaessgroesse_df.head()

,Plan_Fahrt_Id,SITZPLAETZE,KAP_1m2,KAP_2m2,KAP_3m2,KAP_4m2
0,40287,13,15.5,18.0,21.0,23.0
1,60225,13,15.5,18.0,21.0,23.0
2,60178,13,15.5,18.0,21.0,23.0
3,60218,13,15.5,18.0,21.0,23.0
4,60175,13,15.5,18.0,21.0,23.0


Read the CSV file with data about the stops (names of stops, etc.)

In [ ]:
haltestellen_df = pd.read_csv('data/haltestellen.csv', delimiter=';',encoding = "utf-8")
haltestellen_df.head()

,Haltestellen_Id,Haltestellennummer,Haltestellenkurzname,Haltestellenlangname
0,119,1179,HERZ,"Zürich, Herzogenmühlestrasse"
1,104,1186,HEUB,"Zürich, Heubeeriweg"
2,176,1187,HEUR,"Zürich, Heuried"
3,386,6250,HIMM,"Zürich, Himmeri"
4,416,1194,HINT,"Zürich, Hinterbergstrasse"


Read the CSV file with data about the different routes

In [ ]:
linie_df = pd.read_csv('data/linie.csv', delimiter=';',encoding = "utf-8")
linie_df.head()

,Linien_Id,Linienname,VSYS,Linienname_Fahrgastauskunft
0,51,10,T,10
1,36,11,T,11
2,52,12,T,12
3,13,13,T,13
4,14,14,T,14


Read the CSV file with data about the type of day

In [ ]:
tagtyp_df = pd.read_csv('data/tagtyp.csv', delimiter=';',encoding = "utf-8")
tagtyp_df.head()

,Tagtyp_Id,Tagtypname,Bemerkung
0,2,Unbenutzt,NaN
1,7,14-A-23,NaN
2,17,14-B-23,NaN
3,20,14-C-23,NaN
4,32,14-F-23,NaN


Read the main CSV file with data about the passengers and all the links to previous read files

In [ ]:
reisende_df = pd.read_csv('data/reisende.csv', delimiter=';',encoding = "utf-8")
reisende_df.head()

,Tagtyp_Id,Linien_Id,Linienname,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,...,Besetzung,Distanz,Tage_DTV,Tage_DWV,Tage_SA,Tage_SO,Nachtnetz,Tage_SA_N,Tage_SO_N,ID_Abschnitt
0,21,113,15,116279,1,1,161,160.0,09:52:18,9,...,0.44444,289,33.0,0.0,0,33.0,0,0.0,0.0,16100160
1,21,113,15,116279,1,2,160,159.0,09:53:24,9,...,0.88889,535,33.0,0.0,0,33.0,0,0.0,0.0,16000159
2,21,113,15,116279,1,3,159,158.0,09:54:36,9,...,4.77778,282,33.0,0.0,0,33.0,0,0.0,0.0,15900158
3,21,113,15,116279,1,4,158,26.0,09:55:30,9,...,5.88889,424,33.0,0.0,0,33.0,0,0.0,0.0,15800026
4,21,113,15,116279,1,5,26,25.0,09:57:00,9,...,4.22222,258,33.0,0.0,0,33.0,0,0.0,0.0,2600025


# Additional Datasets

To build a geospatial map, we need additional information about the location of the stops. We found data by VBZ (see https://github.com/VerkehrsbetriebeZuerich/vbz-flow-concept/blob/master/data-treatment-jupyter/vbz-jupyter.ipynb).

Unfortunately, this dataset uses different IDs and we cannot merge it using the `Haltestellen_Id`. However, we can map it to the `haltestellen.csv` using the column `Haltestellenlangname`.

In [ ]:
stops = pd.read_csv('data/stops.csv')
stops.head()

,GPS_Latitude,GPS_Longitude,Haltestellen_Id,Haltestellenlangname
0,47.452271,8.571438,595,"Zürich Flughafen, Fracht"
1,47.450239,8.563887,594,"Zürich Flughafen, Bahnhof"
2,47.294990,8.564286,749,"Thalwil, Zentrum"
3,47.370167,8.513776,46,"Zürich, Goldbrunnenplatz"
4,47.437911,8.562140,592,"Glattbrugg, Unterriet"


# Merge Tables

After reading the tables, we merge them in one big table for simpler processing and data analysis.

In [ ]:
# We merge tables into the main table based on the provided ID
data = reisende_df.merge(gefaessgroesse_df, left_on='Plan_Fahrt_Id', right_on='Plan_Fahrt_Id')
data = data.merge(haltestellen_df, left_on='Haltestellen_Id', right_on='Haltestellen_Id')
data = data.merge(linie_df, left_on='Linien_Id', right_on='Linien_Id')
data = data.merge(tagtyp_df, left_on='Tagtyp_Id', right_on='Tagtyp_Id')

# For the geospatial analysis, we are only interested for buses that are going somewhere (not final stop)
data = data.dropna(subset=['Nach_Hst_Id'])
data['Nach_Hst_Id'] = data['Nach_Hst_Id'].astype(int)

# We merge the haltestellen_df again with a different key so that we obtain information about the
# departing stop and the next stop in the same table
data = data.merge(haltestellen_df, left_on='Nach_Hst_Id', right_on='Haltestellen_Id', suffixes=('_from','_to'))

data.head()

<ipython-input-9-814bda7f30a3>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Nach_Hst_Id'] = data['Nach_Hst_Id'].astype(int)


,Tagtyp_Id,Linien_Id,Linienname_x,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id_from,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,...,Haltestellenlangname_from,Linienname_y,VSYS,Linienname_Fahrgastauskunft,Tagtypname,Bemerkung,Haltestellen_Id_to,Haltestellennummer_to,Haltestellenkurzname_to,Haltestellenlangname_to
0,21,113,15,116279,1,1,161,160,09:52:18,9,...,"Zürich, Milchbuck",15,T,15,77-C-23,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse"
1,21,113,15,116969,1,1,161,160,10:02:18,5,...,"Zürich, Milchbuck",15,T,15,77-C-23,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse"
2,21,113,15,114930,1,1,161,160,09:32:18,9,...,"Zürich, Milchbuck",15,T,15,77-C-23,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse"
3,21,113,15,115711,1,1,161,160,09:42:18,6,...,"Zürich, Milchbuck",15,T,15,77-C-23,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse"
4,21,113,15,116353,2,20,159,160,22:44:00,10,...,"Zürich, Berninaplatz",15,T,15,77-C-23,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse"


In [ ]:
# Next, we merge the GPS coordinates into the table
data = data.merge(stops, left_on='Haltestellenlangname_from', right_on='Haltestellenlangname', suffixes=('','_from'))
data = data.merge(stops, left_on='Haltestellenlangname_to', right_on='Haltestellenlangname', suffixes=('','_to'))

# remove duplicates that we produced by merging
data = data.loc[:,~data.columns.duplicated()]
data = data.drop(['Haltestellenlangname'], axis=1)

data.head()

,Tagtyp_Id,Linien_Id,Linienname_x,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id_from,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,...,Bemerkung,Haltestellen_Id_to,Haltestellennummer_to,Haltestellenkurzname_to,Haltestellenlangname_to,GPS_Latitude,GPS_Longitude,Haltestellen_Id,GPS_Latitude_to,GPS_Longitude_to
0,21,113,15,116279,1,1,161,160,09:52:18,9,...,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347
1,21,113,15,116969,1,1,161,160,10:02:18,5,...,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347
2,21,113,15,114930,1,1,161,160,09:32:18,9,...,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347
3,21,113,15,115711,1,1,161,160,09:42:18,6,...,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347
4,20,113,15,118183,1,1,161,160,05:11:12,31,...,NaN,160,1215,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347


In [ ]:
data.columns

Index(['Tagtyp_Id', 'Linien_Id', 'Linienname_x', 'Plan_Fahrt_Id', 'Richtung',
       'Sequenz', 'Haltestellen_Id_from', 'Nach_Hst_Id', 'FZ_AB',
       'Anzahl_Messungen', 'Einsteiger', 'Aussteiger', 'Besetzung', 'Distanz',
       'Tage_DTV', 'Tage_DWV', 'Tage_SA', 'Tage_SO', 'Nachtnetz', 'Tage_SA_N',
       'Tage_SO_N', 'ID_Abschnitt', 'SITZPLAETZE', 'KAP_1m2', 'KAP_2m2',
       'KAP_3m2', 'KAP_4m2', 'Haltestellennummer_from',
       'Haltestellenkurzname_from', 'Haltestellenlangname_from',
       'Linienname_y', 'VSYS', 'Linienname_Fahrgastauskunft', 'Tagtypname',
       'Bemerkung', 'Haltestellen_Id_to', 'Haltestellennummer_to',
       'Haltestellenkurzname_to', 'Haltestellenlangname_to', 'GPS_Latitude',
       'GPS_Longitude', 'Haltestellen_Id', 'GPS_Latitude_to',
       'GPS_Longitude_to'],
      dtype='object')

### Clean Data

The data is already well cleaned. However, we reomve some some connections that we don't want to visualize and round the times to int-values

In [ ]:
# For some reason, the time is shifted by 4h
sorted(reisende_df.FZ_AB.str.slice(stop=2).unique())

['04',
 '05',
 '06',
 '07',
 '08',
 '09',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '20',
 '21',
 '22',
 '23',
 '24',
 '25',
 '26',
 '27',
 '28']

In [ ]:
# therefore, we subtract 4h
data.FZ_AB = data.FZ_AB.str.slice(stop=2).astype(int)
data.FZ_AB -= 4
sorted(data.FZ_AB.unique())

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24]

In [ ]:
# According to the data description, 24:XX:XX corresponds to 0:XX:XX
data.FZ_AB[data.FZ_AB==24] = 0
sorted(data.FZ_AB.unique())

<ipython-input-14-47a9f3c0ddad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.FZ_AB[data.FZ_AB==24] = 0


[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23]

In [ ]:
#Delete connections made by cablecars and night liners
data = data[(data.VSYS == 'B') | (data.VSYS == 'TR') | (data.VSYS == 'T') | (data.VSYS == 'N')]

# Calculate some statistics

Next, we calculate the statistics that we want to visualize in our map

In [ ]:
# create weekday attribute
data['weekday'] = (data.Tage_SA == 0) & (data.Tage_SO == 0) & (data.Nachtnetz == 0) & (data.Tage_SA_N == 0) & (data.Tage_SO_N == 0)

<ipython-input-16-4b103baf3bcb>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['weekday'] = (data.Tage_SA == 0) & (data.Tage_SO == 0) & (data.Nachtnetz == 0) & (data.Tage_SA_N == 0) & (data.Tage_SO_N == 0)


In [ ]:
# Free seats (we clip since Besetzung can be bigger than Sitzplaetze)
data['seat_occupancy'] = data['Besetzung'] / data['SITZPLAETZE']
data['seat_occupancy'] = data['seat_occupancy'].clip(upper=1)
data['free_seats'] = data['SITZPLAETZE'] - data['Besetzung']
data['free_seats'] = data['free_seats'].clip(lower=0)
data.head()

,Tagtyp_Id,Linien_Id,Linienname_x,Plan_Fahrt_Id,Richtung,Sequenz,Haltestellen_Id_from,Nach_Hst_Id,FZ_AB,Anzahl_Messungen,...,Haltestellenkurzname_to,Haltestellenlangname_to,GPS_Latitude,GPS_Longitude,Haltestellen_Id,GPS_Latitude_to,GPS_Longitude_to,weekday,seat_occupancy,free_seats
0,21,113,15,116279,1,1,161,160,5,9,...,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347,False,0.006734,65.55556
1,21,113,15,116969,1,1,161,160,6,5,...,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347,False,0.024242,64.40000
2,21,113,15,114930,1,1,161,160,5,9,...,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347,False,0.011785,65.22222
3,21,113,15,115711,1,1,161,160,5,6,...,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347,False,0.010101,65.33333
4,20,113,15,118183,1,1,161,160,1,31,...,HIRS,"Zürich, Hirschwiesenstrasse",47.397778,8.54175,129,47.400252,8.54347,True,0.003421,65.77419


# Edges for Geospatial Plot

First, we aggregate the data we want to visualize

In [ ]:
connections = data.groupby(['FZ_AB', 'ID_Abschnitt', 'weekday'])
connections = connections.agg(
    seat_occupancy_mean=('seat_occupancy', 'mean'),
    free_seats_max=('free_seats', 'max'),
    free_seats_mean=('free_seats', 'mean'),
    name_from=('Haltestellenlangname_from', 'first'),
    lat_from=('GPS_Latitude', 'first'),
    long_from=('GPS_Longitude', 'first'),
    name_to=('Haltestellenlangname_to', 'first'),
    lat_to=('GPS_Latitude_to', 'first'),
    long_to=('GPS_Longitude_to', 'first')
)

In [ ]:
connections.head()

seat_occupancy_mean  free_seats_max  \
FZ_AB ID_Abschnitt weekday                                        
0     1900466      False               0.012652        31.76923   
      2000411      False               0.048583        31.15385   
      2000412      False               0.129167        53.50000   
                   True                0.034975        58.30303   
      2100022      False               0.070724        31.00000   

                            free_seats_mean              name_from   lat_from  \
FZ_AB ID_Abschnitt weekday                                                      
0     1900466      False          31.595140       Zürich, Balgrist  47.354413   
      2000411      False          30.445345  Zürich, Hegibachplatz  47.361750   
      2000412      False          52.250000  Zürich, Hegibachplatz  47.361750   
                   True           57.901515  Zürich, Hegibachplatz  47.361750   
      2100022      False          29.736840     Zürich, Kreuzplatz  47.365084   

                            long_from                      name_to     lat_to  \
FZ_AB ID_Abschnitt weekday                                                      
0     1900466      False     8.575083             Zürich, Burgwies  47.358070   
      2000411      False     8.560392        Zürich, Signaustrasse  47.362871   
      2000412      False     8.560392         Zürich, Freiestrasse  47.362879   
                   True      8.560392         Zürich, Freiestrasse  47.362879   
      2100022      False     8.554010  Zürich, Bahnhof Stadelhofen  47.366221   

                             long_to  
FZ_AB ID_Abschnitt weekday            
0     1900466      False    8.571704  
      2000411      False    8.557005  
      2000412      False    8.562800  
                   True     8.562800  
      2100022      False    8.548049

In [ ]:
# Remove invalid data (this happens when there are no measurements for some connections)
connections = connections[pd.notna(connections['free_seats_max'])]
connections = connections[pd.notna(connections['free_seats_mean'])]

# Convert to Integer
connections['free_seats_max'] = connections['free_seats_max'].astype(int)
connections['free_seats_mean'] = connections['free_seats_mean'].astype(int)
connections['seat_occupancy_mean'] = connections['seat_occupancy_mean'] * 100
connections['seat_occupancy_mean'] = connections['seat_occupancy_mean'].astype(int)

# Reset index (otherwise, FZ_AB is the index)
connections = connections.reset_index()

# Remove all lines, where start and ziel are identical
connections = connections.query("lat_from != lat_to and long_from != long_to")

connections.head()

,FZ_AB,ID_Abschnitt,weekday,seat_occupancy_mean,free_seats_max,free_seats_mean,name_from,lat_from,long_from,name_to,lat_to,long_to
0,0,1900466,False,1,31,31,"Zürich, Balgrist",47.354413,8.575083,"Zürich, Burgwies",47.358070,8.571704
1,0,2000411,False,4,31,30,"Zürich, Hegibachplatz",47.361750,8.560392,"Zürich, Signaustrasse",47.362871,8.557005
2,0,2000412,False,12,53,52,"Zürich, Hegibachplatz",47.361750,8.560392,"Zürich, Freiestrasse",47.362879,8.562800
3,0,2000412,True,3,58,57,"Zürich, Hegibachplatz",47.361750,8.560392,"Zürich, Freiestrasse",47.362879,8.562800
4,0,2100022,False,7,31,29,"Zürich, Kreuzplatz",47.365084,8.554010,"Zürich, Bahnhof Stadelhofen",47.366221,8.548049


Export the Stops as Geojson (see https://github.com/VerkehrsbetriebeZuerich/vbz-flow-concept/blob/master/data-treatment-jupyter/vbz-jupyter.ipynb)

In [ ]:
connections_coord = connections.copy()
length_param = 0.0005 #shorten vector to make more readable

for index,row in connections_coord.iterrows():

    dlong = row[-1]-row[-4]
    dlat = row[-2]-row[-5]

    #section of traffic flow
    vector = (dlat, dlong)

    #scalar length of vector
    length = math.sqrt(dlat**2 + dlong**2)
    unitvector = (dlat/length,dlong/length)

    connections_coord.loc[index, 'lat_from'] = row[-5] + (length_param * unitvector[0])
    connections_coord.loc[index, 'long_from'] = row[-4] + (length_param * unitvector[1])
    connections_coord.loc[index, 'lat_to'] = row[-2] - (length_param * unitvector[0])
    connections_coord.loc[index, 'long_to'] = row[-1] - (length_param * unitvector[1])

Output it to a GeoJson file

In [ ]:
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as mcolors

# Later, we will draw each line as a color. Here, we define a mapping from seat occupancy to a color
# Inserting the color already here in the JSON allows better visualization later on...
norm = mpl.colors.Normalize(vmin=0, vmax=100)
cmap = cm.coolwarm
color_mapper = cm.ScalarMappable(norm=norm, cmap=cmap)


#define function to manually output geojson file (slimmer than geopandas creates its files!)
proctable=connections_coord.copy()
def df_to_geojson(proctable, properties, lat='latitude', lon='longitude'):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in proctable.iterrows():
        feature = {'type':'Feature',
                   'properties':{'stroke': mcolors.to_hex(color_mapper.to_rgba(row['seat_occupancy_mean']))},
                   'geometry':{'type':'LineString',
                               'coordinates':[[]]}}
        feature['geometry']['coordinates'] = [[row['long_from'],row['lat_from']], [row['long_to'],row['lat_to']]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

#define attributes to be included in geojson file
col = ['FZ_AB', 'weekday', 'seat_occupancy_mean', 'free_seats_max', 'free_seats_mean', 'name_from', 'name_to']

#output
geojson = df_to_geojson(proctable, col)
output_filename = "vbz.geojson"
with open(output_filename, 'w', encoding='utf-8') as output_file:
    json.dump(geojson, output_file, separators=(", ", ": "), ensure_ascii=False)

# Visualize the Data

In [ ]:
vbz_geo = gpd.read_file('vbz.geojson')

In [ ]:
vbz_geo.head()

,stroke,FZ_AB,weekday,seat_occupancy_mean,free_seats_max,free_seats_mean,name_from,name_to,geometry
0,#3d50c3,0,False,1,31,31,"Zürich, Balgrist","Zürich, Burgwies","LINESTRING (8.57474 47.35478, 8.57204 47.35770)"
1,#465ecf,0,False,4,31,30,"Zürich, Hegibachplatz","Zürich, Signaustrasse","LINESTRING (8.55992 47.36191, 8.55748 47.36271)"
2,#5f7fe8,0,False,12,53,52,"Zürich, Hegibachplatz","Zürich, Freiestrasse","LINESTRING (8.56084 47.36196, 8.56235 47.36267)"
3,#4358cb,0,True,3,58,57,"Zürich, Hegibachplatz","Zürich, Freiestrasse","LINESTRING (8.56084 47.36196, 8.56235 47.36267)"
4,#4f69d9,0,False,7,31,29,"Zürich, Kreuzplatz","Zürich, Bahnhof Stadelhofen","LINESTRING (8.55352 47.36518, 8.54854 47.36613)"


In [ ]:
vbz_geo.FZ_AB.unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

We create a map using `folium` and directly draw on it the data using the previously generated json.

In [ ]:
def get_map(time, weekday):
  zurich_map = folium.Map(location=[47.38, 8.55], zoom_start=13)
  vbz_geo_show = vbz_geo[(vbz_geo.FZ_AB == int(time)) & (vbz_geo.weekday == weekday)]

  folium.GeoJson(
      vbz_geo_show.to_json(),
      name="VBZ Data",
      control=True,
      style_function = lambda x: {'color' : x['properties']['stroke'], 'opacity' : 0.7, 'weight': 3.0},
      highlight_function = lambda x: {'opacity': 0.9, 'weight': 6.0},
      tooltip=folium.features.GeoJsonTooltip(
          fields=['name_from', 'name_to', 'seat_occupancy_mean', 'free_seats_mean'],
          aliases=['From: ', 'To: ', 'Average Seat Occupation[%]: ', 'Average Free Seats: '],
          style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;") # setting style for popup box
        )
      ).add_to(zurich_map)

  folium.LayerControl().add_to(zurich_map)

  return zurich_map

time = input("Please enter a time [0-24]")
weekday = input("Weekday [yes/no]?")
weekday = (weekday.lower() == "yes")

zurich_map = get_map(time, weekday)

Please enter a time [0-24]23
Weekday [yes/no]?yes
0


Save the map as HTML

In [ ]:
# zurich_map.save(f'map_{time}_{weekday}.html')

AttributeError: 'Map' object has no attribute 'display'

Create all maps that later can be used for visualization

In [ ]:
for time in range(26):
  for weekday in [True, False]:
    zurich_map = get_map(time, True)
    try:
      zurich_map.save(f'result/map_{time}_{weekday}.html')
    except IndexError:
      print(f"No data for {time} {weekday}")

165
165
864
864
918
918
917
917
918
918
918
918
915
915
918
918
920
920
918
918
915
915
921
921
920
920
917
917
918
918
916
916
912
912
885
885
884
884
821
821
798
798
128
128
0
No data for 22 True
0
No data for 22 False
0
No data for 23 True
0
No data for 23 False
0
No data for 24 True
0
No data for 24 False
0
No data for 25 True
0
No data for 25 False


In [ ]:
import shutil
shutil.make_archive('maps', 'zip', 'result/')

'/content/maps.zip'